# 2D Phase Diagram

In [ ]:
import mmf_setup;mmf_setup.nbinit()
%pylab inline --no-import-all
from nbimports import * 

## A FF State point
mu, g, dmu, dq, delta, na, nb
* Pick a FF state
* $\mu=0.5$
* g=-0.46649458220982054
* $d\mu=0.00039283710065919316$ 
* $dq=7.856742013183863e-05$
* $\Delta=0.0007071067811865476$
* $n_a=0.07957751184849551$
* $n_b=0.07957751184849551$)

In [ ]:
from mmf_hfb import FuldeFerrelState; reload(FuldeFerrelState)
qs = [0.5]#np.linspace(0,3,4) # q does not change the pressure
dq0 = 7.856742013183863e-5
mu=0.5
dmu = 0.00039283710065919316
delta = 0.0007071067811865476
for q in qs:
    dqs = np.linspace(2,7,30)*dq0
    ff = FuldeFerrelState.FFState(mu=mu, dmu=dmu, d=2, delta=delta, k_c=500)
    ps = [ff.get_pressure(mu=mu, dmu=dmu,q = q, dq=dq).n for dq in dqs]
    clear_output()
    plt.plot(dqs * 1000, ps)

## Load Json Data
* To compute the phase diagram inside the notebook is not pratical as it's very computationally expensive. The job is done in a multi-process pool routine which generates data that can be loaded and present here.

In [ ]:
import sys
import os
import inspect
from os.path import join
import json
from json import dumps
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
files = ["3d_phase_map_data.txt"]
#for i in [2]: # To support multiple files
#    files.append(f"2d_phase_map_data_0{i}.txt")
rets = []
for file in files:
    file = join(currentdir,"../",file)
    if os.path.exists(file):
        with open(file,'r') as rf:
            rets.extend(json.load(rf))
files

### Data format
* The overall data is a list, each item in the list is a dict with 3 entries,  **mu** and **delta**($\Delta_0$), and **data**
* The format of the **data** is a list of array, each array is a tuple of 5 numbers,  corresond to: ($d\mu$, dq, g, $\Delta$, pressure). The $\Delta$ is the new value when dq and dmu are not zero, so it may be different from the $\Delta$ in the dict.



#### The piece of code in the following cell find the maximum pressure for given(mu, $\Delta_0$, dmu), which can determine if dq is zero or not

In [ ]:
formattedData = []
#find the dq with maximum pressure
for items in rets:
    mu = items['mu']
    delta = items['delta']
    data = items['data']
    if len(data) > 0:
        ret = data[0]
        dmu,dq, g, d, p = ret
        for item in data:
            if dmu == item[0]:
                if item[4] > p:
                    dq = item[1]
                    p = item[4]
            else:
                formattedData.append((mu, g, dmu, dq, delta))
                dmu, dq, g, d, p = item
        formattedData.append((mu, g, dmu, dq, delta))

### Compute density
* Take couple of minutes to compute the density

In [ ]:
ff = FuldeFerrelState.FFState(fix_g=True,k_c=100, d=3)
data = []
for i in range(len(formattedData)):
    if len(formattedData[i]) > 5:
        mu, g, dmu, dq, delta,na, nb = formattedData[i]
    else:
        mu, g, dmu, dq, delta = formattedData[i]
    na, nb = ff.get_densities(mu=mu, dmu=0, delta=delta)
    data.append(formattedData[i][:5] + (na.n, nb.n,))


### Plot the phase diagram

In [ ]:
plt.figure(figsize=(20, 5))
plt.subplot(121)
x,y,c,area =[],[],[],[]
for item in data:
    n = item[5] + item[6]
    x.append(-1.0/(item[1] * n**(1.0/3)))
    y.append(item[2]/item[4])
    if item[2] > 0 and item[3] > 0:
        c.append('r')
        area.append(16)
    else:
        c.append('b')
        area.append(1)
plt.scatter(x, y, s=area, c=c)
plt.xlabel(r"$-1/(g\sqrt{n_{d\mu=0}})$", fontsize=20)
plt.ylabel(r"$d\mu/\Delta_0$", fontsize=20)
plt.subplot(122)
x,y=[],[]
for item in data:
    x.append(item[4])
    y.append(item[2]/item[4])

plt.scatter(x, y, s=area, c=c)
plt.xlabel(r"$\mu$", fontsize=20)
plt.ylabel(r"$d\mu/\Delta_0$", fontsize=20)

Pressue is a function of $\Delta$, $\mu$, $d\mu$, q, and dq:

$$
\newcommand{\E}{\mathcal{E}}
P(\Delta, \mu,d\mu, q, dq)= n_a(\Delta, \mu,d\mu, q, dq) \mu_a + n_b(\Delta, \mu,d\mu, q, dq) \mu_b -\E(\Delta, \mu,d\mu, q, dq)
$$
where 

$$
\E = \kappa\\
n_a = \frac{n_+ + n_-}{2} \qquad
n_b = \frac{n_+ - n_-}{2}\qquad
\mu_a = \frac{\mu_+ + \mu_-}{2}\qquad
\mu_b = \frac{\mu_+ - \mu_-}{2}\qquad
f_{\beta}(E) = \frac{1}{1+e^{\beta E}}.
$$

\begin{align}
\newcommand\intdk{\int\frac{\d^d{k}}{(2\pi)^d}\;}
\newcommand\q {\partial dq}
  n_+ &= \intdk\overbrace{\left(1 - \frac{\epsilon^+_k}{E_k}
               \underbrace{\bigl(f(\omega_-) - f(\omega_+)\bigr)}_{f_\nu}
          \right)}^{f_+}, &
  n_- &= \intdk\overbrace{\bigl(f(\omega_+) - f(-\omega_-)\bigr)}^{f_-},\\
  \tau_+ &= \intdk k^2f_+, &
  \tau_- &= \intdk k^2f_-,\\
  \nu &= -\intdk\frac{\Delta}{2E_k}\overbrace{\bigl(f(\omega_-)-f(\omega_+)\bigr)}^{f_\nu}, &
  \kappa &= \intdk \left(\frac{\hbar^2\tau_+}{2m_+} - \frac{\abs{\Delta}^2}{2E_k}f_\nu\right),\\
\end{align}

Using the chain rule, we may be able to find the condition that maximize the pressure:
\begin{align}
\frac{\partial P}{\q} &= 0\\
&=\frac{\partial n_a}{\q}\mu_a + \frac{\partial n_b}{\q}\mu_b -\frac{\partial \kappa}{\q}\\
&=\frac{\partial (n_+ + n_-)}{4\q}(\mu_+ + \mu_-) + \frac{\partial (n_+ - n_-)}{4\q}(\mu_+ - \mu_-) -\frac{\partial \kappa}{\q}\\
&=\frac{\partial n_+}{2\q}\mu_+ + \frac{\partial n_-}{2\q}\mu_- -\frac{\partial \kappa}{\q}
\end{align}
With this we can perform chain rules:

\begin{align}
k_a = k + q + dq, \qquad k_b = k+q - dq \\
\epsilon_a = \frac{\hbar^2}{2m}k_a^2 - \mu_a, \qquad \epsilon_b = \frac{\hbar^2}{2m}k_b^2 - \mu_b\\
E=\sqrt{\epsilon_+^2+\abs{\Delta}^2},\qquad \omega_+= \epsilon_-+E, \qquad \omega_- = \epsilon_- - E
\end{align}
\begin{align}
\epsilon_+
&= \frac{\hbar^2}{4m}(k_a^2+k_b^2) - \mu_+\\
&= \frac{\hbar^2}{2m}\left[(k+q)^2 + dq^2\right] - \mu_+\\
\epsilon_- 
&= \frac{\hbar^2}{4m}(k_a^2-k_b^2) - \mu_-\\
&=\frac{\hbar^2}{m}(k +q)dq - \mu_-\\
\end{align}

\begin{align}
\frac{\partial \epsilon_+}{\q}
&= \frac{\hbar^2 dq}{m}\\
\frac{\partial E}{\q}
&= \frac{\epsilon_+}{\sqrt{\epsilon_+^2 + \abs{\Delta}}}\frac{\partial \epsilon_+}{\q}
\end{align}

\begin{align}
\frac{\partial n+}{\q}
&=\intdk\left[ \frac{\partial}{\q}\left(\frac{\epsilon_+}{E}\right)f_{\nu} + \frac{\epsilon_+}{E} \frac{\partial f_{\nu}}{\q}\right]\\
&=\intdk\left[\left( \frac{\partial \epsilon_+}{E\q}-\frac{ \epsilon_+\partial E}{E^2\q}\right)f_{\nu} + \frac{\epsilon_+}{E} \frac{\partial}{\q}f_{\nu}\right]\\
&=\intdk\left[\left(\frac{\hbar^2 dq}{mE} -\frac{\epsilon_+^2}{E^2\sqrt{\epsilon_+^2 + \abs{\Delta}}}
\right)\frac{\partial \epsilon_+}{\q}f_{\nu} + \frac{\epsilon_+}{E} \frac{\partial}{\q}f_{\nu}\right]
\end{align}

\begin{align}
\frac{\partial n_-}{\q}
&=\intdk\left[ \frac{\partial}{\q}f(\omega_+)- \frac{\partial}{\q}f(-\omega_-)\right]
\end{align}
\begin{align}
  \frac{\partial \kappa}{\q} &= \intdk \left[\frac{\hbar^2}{2m_+}\frac{\partial \tau_+}{\q} - \abs{\Delta}^2\frac{\partial}{\q}\frac{f_\nu}{2E}\right],\\
\end{align}

$$
f_+ = 1 - \frac{\epsilon_+}{E}\left(f(\omega_-)-f(\omega_+)\right)\\
f_-=f(\omega_+)-f(-\omega_-)\\
f_a = \frac{f_++f_-}{2}\\
f_b = \frac{f_+ - f_-}{2}\\
$$

\begin{align}
\frac{\partial f_+}{\q}
&=-\left[\frac{\partial}{\q}\left(\frac{\epsilon_+}{E}\right)\left(f(\omega_-)-f(\omega_+)\right)+
\frac{\epsilon_+}{E}\overbrace{\frac{\partial\left(f(\omega_-)-f(\omega+)\right) }{\q}}^0
\right]\\
&=-\left[\frac{\partial}{\q}\left(\frac{\epsilon_+}{E}\right)\left(f(\omega_-)-f(\omega_+)\right)
\right]\\
\end{align}

\begin{align}
\tau_+ 
&= k_a^2 f_a + k_b^2  f_b\\
&=\left[(k+q)^2 + dq^2\right] f_+ + 2(k+q)dqf_-\\
\frac{\partial \tau_+}{\q}
&= 2dq f_+ + \left[(k+q)^2 + dq^2\right]\frac{\partial f_+}{\q} + 2(k+q)f_- + 2(k+q)dq\overbrace{\frac{\partial f_-}{\q}}^0\\
&= 2dq f_+ - \left[(k+q)^2 + dq^2\right]\left[\frac{\partial}{\q}\left(\frac{\epsilon_+}{E}\right)\left(f(\omega_-)-f(\omega_+)\right)\right] + 2(k+q)f_- \\\end{align}

When $T=0$, $f(x)$ is a step function, its derviative is zero except at $x=0$:

$$
\frac{\partial f_{\nu}}{\q}=\frac{\partial f}{\q}=0
$$

Then
\begin{align}
\frac{\partial n_-}{\q}
&=\intdk\left[ \frac{\partial}{\q}f(\omega_+)- \frac{\partial}{\q}f(-\omega_-)\right]=0\\
\frac{\partial n_+}{\q}
&=\intdk\left[ \frac{\partial}{\q}\left(\frac{\epsilon_+}{E}\right)f_{\nu} + \frac{\epsilon_+}{E} \frac{\partial f_{\nu}}{\q}\right]\\
&=\intdk \frac{\partial}{\q}\left(\frac{\epsilon_+}{E}\right)f_{\nu}
\end{align}

\begin{align}
\frac{\partial \tau_+}{\q}
&= 2dq f_+ + \left[(k+q)^2 + dq^2\right]\frac{\partial f_+}{\q} + 2(k+q)f_-
\end{align}

\begin{align}
 \kappa &= \intdk \left(\frac{\hbar^2\tau_+}{2m_+} - \frac{\abs{\Delta}^2}{2E_k}f_\nu\right),\\
 \frac{\partial \kappa}{\q}
 &= \intdk \left(\frac{\hbar^2}{2m_+}\frac{\partial \tau_+}{\q}+ \frac{\abs{\Delta}^2\partial E}{2E^2\q}f_\nu - \frac{\abs{\Delta}^2}{2E_k}\overbrace{\frac{\partial f_\nu}{\q}}^0\right),\\
 &= \intdk \left(\frac{\hbar^2}{2m_+}\frac{\partial \tau_+}{\q}+ \frac{\abs{\Delta}^2\partial E}{2E^2\q}f_\nu \right),\\ 
 &= \intdk \left(\frac{\hbar^2}{2m_+}\left(2dq f_+ - \left[(k+q)^2 + dq^2\right]\left[\frac{\partial}{\q}\left(\frac{\epsilon_+}{E}\right)\left(f(\omega_-)-f(\omega_+)\right)\right] + 2(k+q)f_-\right)+ \frac{\abs{\Delta}^2\partial E}{2E^2\q}f_\nu \right),\\
  &= \intdk \left[\frac{\hbar^2}{2m}\left(2dq f_+ + 2(k+q)f_- - \left[(k+q)^2 + dq^2\right]f_{\nu}\frac{\partial}{\q}\left(\frac{\epsilon_+}{E}\right) \right)+ \frac{\abs{\Delta}^2\partial E}{2E^2\q}f_\nu \right],\\
 \end{align}